# Image Classification (Transfer Learning) 모델 제작
EfficientNet : https://greeksharifa.github.io/computer%20vision/2022/03/01/EfficientNet/

In [2]:
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=39e9891297d1bb5526669f1fbc3af24a6bb0b5eeb248d1c3b0ae08648f0ae005
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import time
import copy
from tqdm.notebook import tqdm

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from torch.autograd import Variable
from torch.utils.data import Subset

from torchvision import transforms, datasets

from efficientnet_pytorch import EfficientNet

In [23]:
model = EfficientNet.from_pretrained('efficientnet-b0') 
model_name = 'efficientnet-b0'

model = EfficientNet.from_pretrained(model_name, num_classes=3) 

batch_size  = 8 
random_seed = 247
torch.manual_seed(random_seed)

data_path = '/content/drive/MyDrive/dataset' 
dataset = datasets.ImageFolder(data_path, transforms.Compose([transforms.Resize((224, 224)),  
                                                              transforms.ToTensor(), 
                                                              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) , ]))


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [6]:
dataset.class_to_idx

{'paper': 0, 'rock': 1, 'scissors': 2}

In [20]:
train_idx, tmp_idx = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=random_seed) 
datasets = {}

datasets['train'] = Subset(dataset, train_idx) 
datasets['valid'] = Subset(dataset, tmp_idx) 


dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],batch_size=batch_size, 
                                                   shuffle=True,num_workers=1)

dataloaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],batch_size=batch_size, 
                                                   shuffle=False,num_workers=1)

batch_num['train'], batch_num['valid'] = len(dataloaders['train']), len(dataloaders['valid'])
print('batch_size : %d,  tv : %d / %d' % (batch_size, batch_num['train'], batch_num['valid']))

batch_size : 8,  tv : 308 / 77


In [19]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train() 
            else:
                model.eval() 

            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)


                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)

            if phase == 'train':
                scheduler.step() 
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            
            if phase == 'train': 
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else: 
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
            print('{} Loss: {:.2f} Acc: {:.1f}'.format(phase, epoch_loss, epoch_acc))
           
            if phase == 'valid' and epoch_acc > best_acc: 
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

    
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'rps_model.pt')
    print('model saved')
    
    return model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  

criterion = nn.CrossEntropyLoss() 

optimizer_ft = optim.SGD(model.parameters(),  
                         lr = 1e-4,
                         momentum=0.9,
                         weight_decay=1e-4)

lmbda = lambda epoch: 0.98739
exp_lr_scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer_ft, lr_lambda=lmbda) 

In [10]:
model = model.to(device)

# 모델 학습

In [11]:
model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, criterion, optimizer_ft, 
                                                                                        exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/308 [00:00<?, ?it/s]

train Loss: 1.04 Acc: 50.8


  0%|          | 0/77 [00:00<?, ?it/s]

valid Loss: 0.95 Acc: 68.1
==> best model saved - 0 / 68.1
Epoch 1/9
----------


...

valid Loss: 0.28 Acc: 94.0
==> best model saved - 8 / 94.0
Epoch 9/9
----------


  0%|          | 0/308 [00:00<?, ?it/s]

train Loss: 0.33 Acc: 90.2


  0%|          | 0/77 [00:00<?, ?it/s]

valid Loss: 0.23 Acc: 94.5
==> best model saved - 9 / 94.5
Training complete in 36m 50s
Best valid Acc: 9 - 94.5
model saved


# 테스트

In [12]:
from PIL import Image
import torch
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3)
model.load_state_dict(torch.load('/content/rps_model.pt', map_location=torch.device("cpu")))
model.eval()

device = torch.device("cpu")

Loaded pretrained weights for efficientnet-b0


In [13]:
image = (lambda x : transforms.Compose([transforms.Resize((224, 224)), \
          transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) \
          (Image.open(x)).unsqueeze(0))('/content/rock.jpg')

label = {0: 'paper', 1: 'rock', 2: 'scissors'}

In [17]:
with torch.no_grad():
  _, preds = torch.max(model(image.to(device)), 1) 

print(label[preds.cpu().numpy()[0]])

rock


# 가위바위보 게임

In [25]:
import time
import random

from PIL import Image
import torch
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

my_score = 0 
computer_score = 0 

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3)
model.load_state_dict(torch.load('/content/rps_model.pt', map_location=torch.device("cpu")))
model.eval()

device = torch.device("cpu")

label = {0: 'paper', 1: 'rock', 2: 'scissors'}
win = {2: 0, 0: 1, 1: 2} 

Loaded pretrained weights for efficientnet-b0


In [28]:
input_image = input("이미지 경로를 입력해주세요 : ")

image = (lambda x : transforms.Compose([transforms.Resize((224, 224)), \
          transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) \
          (Image.open(x)).unsqueeze(0))(input_image)

with torch.no_grad():
  _, result = torch.max(model(image.to(device)), 1)

img_res = result.cpu().numpy()[0]

random_rps = random.randint(0, 2) 
print("가위 바위 보 게임을 시작합니다.")

print(f"이미지의 손 모양 : {label[img_res]}")
print(f"컴퓨터의 선택 : {label[random_rps]}")
print()

if img_res == random_rps:
  print("비겼습니다!")

  my_score += 1
  computer_score += 1

  print()
  print("현재 점수") 
  print(f"내 점수 : {my_score}") 
  print(f"컴퓨터 점수 : {computer_score}") 

elif win[img_res] == random_rps: 
  print("이겼습니다!")

  my_score += 3

  print()
  print("현재 점수") 
  print(f"내 점수 : {my_score}") 
  print(f"컴퓨터 점수 : {computer_score}") 
else: 
  print("졌습니다!")

  computer_score += 3

  print()
  print("현재 점수") 
  print(f"내 점수 : {my_score}") 
  print(f"컴퓨터 점수 : {computer_score}") 

이미지 경로를 입력해주세요 : rock.jpg
가위 바위 보 게임을 시작합니다.
이미지의 손 모양 : rock
컴퓨터의 선택 : rock

비겼습니다!

현재 점수
내 점수 : 2
컴퓨터 점수 : 5
